In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
import torch
import pandas as pd
!pip install transformers

Found GPU at: /device:GPU:0
     |████████████████████████████████| 3.4 MB 5.5 MB/s 
     |████████████████████████████████| 596 kB 45.8 MB/s 
     |████████████████████████████████| 895 kB 39.4 MB/s 
     |████████████████████████████████| 61 kB 550 kB/s 
     |████████████████████████████████| 3.3 MB 24.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [4]:
from google.colab import drive
drive.mount('/content/drive')
train_filename = "/content/drive/MyDrive/Colab Notebooks/IronyDatasets/train_dataset.txt"
test_filename = "/content/drive/MyDrive/Colab Notebooks/IronyDatasets/val_dataset.txt"
df_train = pd.read_csv(train_filename, delimiter='\t', header=None, names=['label', 'tweet'], skiprows=1,usecols=[1,2])
df_test = pd.read_csv(test_filename, delimiter='\t', header=None, names=['label', 'tweet'], skiprows=1,usecols=[1,2])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import re 

def preprocess(tweet):
    # Convert www.* or https?://* to 'link'
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','link',tweet)
    # Separate hashtags into separate words
    tweet = ' '.join(re.split(r'(?=[A-Z])', tweet))
    # Convert @username to 'username'
    tweet = re.sub('@[^\s]+','username',tweet)
    # Convert to lower case
    tweet = tweet.lower()
    #Convert '#' to 'hash'
    tweet = tweet.replace('#','hash ')
    # Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    return tweet

num_train_tweets = df_train['tweet'].count()
for i in range(num_train_tweets):
    df_train.at[i,'tweet'] = preprocess(df_train['tweet'][i])

num_val_tweets = df_test['tweet'].count()
for i in range(num_val_tweets):
    df_test.at[i,'tweet'] = preprocess(df_test['tweet'][i])

In [ ]:
train_tweets = df_train.tweet.values
train_labels = df_train.label.values

test_tweets = df_test.tweet.values
test_labels = df_test.label.values

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
train_input_ids = []
test_input_ids = []

for tweet in train_tweets:
    encoded_tweet = tokenizer.encode(tweet, add_special_tokens = True)
    train_input_ids.append(encoded_tweet)

for tweet in test_tweets:
    encoded_tweet = tokenizer.encode(tweet, add_special_tokens = True)
    test_input_ids.append(encoded_tweet)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

MAX_LEN = 350
train_input_ids = pad_sequences(train_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")

In [ ]:
train_attention_masks = []
test_attention_masks = []

for tweet in train_input_ids:
    att_mask = [int(token_id > 0) for token_id in tweet]
    train_attention_masks.append(att_mask)

for tweet in test_input_ids:
    att_mask = [int(token_id > 0) for token_id in tweet]
    test_attention_masks.append(att_mask)

In [ ]:
# Convert all inputs and labels into torch tensors, the required datatype for our model.
train_inputs = torch.tensor(train_input_ids)
test_inputs = torch.tensor(test_input_ids)

train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

train_masks = torch.tensor(train_attention_masks)
test_masks = torch.tensor(test_attention_masks)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 16

# Create DataLoaders
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

model.cuda()

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup
epochs = 4
# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
import random

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack batch
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        # Returns loss
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        loss = outputs[0]

        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and learning rate
        optimizer.step()
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # Validation

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in test_dataloader:

        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        
        # Get the "logits" output by the model, which are the values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch    40  of    239.    Elapsed: 0:01:13.
  Batch    80  of    239.    Elapsed: 0:02:27.
  Batch   120  of    239.    Elapsed: 0:03:40.
  Batch   160  of    239.    Elapsed: 0:04:53.
  Batch   200  of    239.    Elapsed: 0:06:06.

  Average training loss: 0.65
  Training epcoh took: 0:07:17

Running Validation...
  Accuracy: 0.70
  Validation took: 0:00:34

======== Epoch 2 / 4 ========
Training...
  Batch    40  of    239.    Elapsed: 0:01:13.
  Batch    80  of    239.    Elapsed: 0:02:26.
  Batch   120  of    239.    Elapsed: 0:03:39.
  Batch   160  of    239.    Elapsed: 0:04:52.
  Batch   200  of    239.    Elapsed: 0:06:05.

  Average training loss: 0.54
  Training epcoh took: 0:07:16

Running Validation...
  Accuracy: 0.71
  Validation took: 0:00:34

======== Epoch 3 / 4 ========
Training...
  Batch    40  of    239.    Elapsed: 0:01:13.
  Batch    80  of    239.    Elapsed: 0:02:26.
  Batch   120  of    239.    Elapsed: 0:03:39.
  

In [ ]:
# Convert to tensors.
prediction_inputs = torch.tensor(test_input_ids)
prediction_masks = torch.tensor(test_attention_masks)
prediction_labels = torch.tensor(test_labels)

model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in test_dataloader:
  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_labels = batch
  with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


    DONE.


In [ ]:
TP = 0
TN = 0
FP = 0
FN = 0

for i in range(len(true_labels)):
  pred_labels = np.argmax(predictions[i], axis=1).flatten()
  for j in range(batch_size):
    if pred_labels[j] == 1:
      if true_labels[i][j] ==1:
        TP += 1
      else:
        FP += 1
    else:
      if true_labels[i][j] == 1:
        FN += 1
      else:
        TN += 1

accuracy = (TP + TN)/(TP + FP + TN + FN)
print('Accuracy:')
print(accuracy)
print(' ')

precision = TP/(TP + FP)
print('Precision:')
print(precision)
print(' ')

recall = TP/(TP + FN)
print('Recall:')
print(recall)
print(' ')

F1 = 2*precision*recall/(precision + recall)
print('F1 Score:')
print(F1)

Accuracy:
0.7028061224489796
 
Precision:
0.5979899497487438
 
Recall:
0.7652733118971061
 
F1 Score:
0.6713681241184768
